# DSC 210 Final Course Project: Image Compression 
Team members:
- Shreya Reddy Pakala
- Yashi Shukla 
- Sai Kaushik Soma
- Sagarika Sardesai

In [1]:
#importing required libraries
import os
import cv2 
import warnings
import numpy as np
import scipy.fftpack
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab

from matplotlib import rc
from numpy.linalg import svd
from scipy import signal,misc
from math import pi, cos, sqrt, log10
from cv2 import imread, imwrite, imshow
from numpy import pi,sin,r_,zeros, array, clip, trunc
from skimage.metrics import structural_similarity as ssim
from ipywidgets import interact,interactive,interact_manual

warnings.filterwarnings('ignore')

%matplotlib inline
pylab.rcParams['figure.figsize'] = (20.0, 7.0)

# SVD

In [2]:
def compress_svd(image,k):
    """
    Perform svd decomposition and truncated (using k singular values/vectors) reconstruction
    returns
    --------
      reconstructed matrix reconst_matrix, array of singular values s
    """
    U,s,V = svd(image,full_matrices=False)
    reconst_matrix = np.dot(U[:,:k],np.dot(np.diag(s[:k]),V[:k,:]))
   
    return reconst_matrix

# DCT

In [3]:
def dct2(a):
    return scipy.fftpack.dct( scipy.fftpack.dct( a, axis=0, norm='ortho' ), axis=1, norm='ortho' )

def idct2(a):
    return scipy.fftpack.idct( scipy.fftpack.idct( a, axis=0 , norm='ortho'), axis=1 , norm='ortho')

def compress_dct(im,thresh):
    imsize = im.shape
    dct = np.zeros(imsize)

    # Do 8x8 DCT on image (in-place)
    for i in r_[:imsize[0]:8]:
        for j in r_[:imsize[1]:8]:
            dct[i:(i+8),j:(j+8)] = dct2( im[i:(i+8),j:(j+8)] )
            
    # Threshold
    dct_thresh = dct * (abs(dct) > (thresh*np.max(dct)))
    
    percent_nonzeros = np.sum( dct_thresh != 0.0 ) / (imsize[0]*imsize[1]*1.0)
    
    im_dct = np.zeros(imsize)

    for i in r_[:imsize[0]:8]:
        for j in r_[:imsize[1]:8]:
            im_dct[i:(i+8),j:(j+8)] = idct2( dct_thresh[i:(i+8),j:(j+8)] )
            
    return im_dct

# Performance Metrics

In [4]:
def compressionratio(original_img_arr, compressed_img_arr):
    
    cv2.imwrite(f"4kImages/original_temp.jpg", original_img_arr)
    original_size = round(os.stat("4kImages/original_temp.jpg").st_size)
    
    cv2.imwrite(f"4kImages/compressed_temp.jpg", compressed_img_arr)
    compressed_size = round(os.stat("4kImages/compressed_temp.jpg").st_size)

    compression_ratio = original_size/compressed_size

    return round(compression_ratio,2),original_size,compressed_size

In [5]:
def get_mse(original_img_arr, decoded_img_arr):
    mse = np.sum((original_img_arr.astype("float") - decoded_img_arr.astype("float")) ** 2)
    mse /= float(original_img_arr.shape[0] * decoded_img_arr.shape[1])
    return mse

In [6]:
def get_psnr(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    if(mse == 0):  # MSE is zero means no noise is present in the signal ==> PSNR has no importance.
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

In [7]:
def get_ssim(img1, img2):
    if not img1.shape == img2.shape:
        raise ValueError('Input images must have the same dimensions.')
    if img1.ndim == 2:
        return ssim(img1, img2)
    elif img1.ndim == 3:
        if img1.shape[2] == 3:
            ssims = []
            for i in range(3):
                ssims.append(ssim(img1, img2))
            return np.array(ssims).mean()
        elif img1.shape[2] == 1:
            return ssim(np.squeeze(img1), np.squeeze(img2))
    else:
        raise ValueError('Wrong input image dimensions.')
        

# Getting the images to perform compression

In [8]:
data_images = {
    "Image1":imread("4kImages/4k-3840-x-2160-wallpapers-themefoxx (1).jpg",cv2.IMREAD_GRAYSCALE),
    "Image2":imread("4kImages/4k-3840-x-2160-wallpapers-themefoxx (2).jpg",cv2.IMREAD_GRAYSCALE),
    "Image3":imread("4kImages/4k-3840-x-2160-wallpapers-themefoxx (3).jpg",cv2.IMREAD_GRAYSCALE),
    "Image4":imread("4kImages/4k-3840-x-2160-wallpapers-themefoxx (4).jpg",cv2.IMREAD_GRAYSCALE),
    "Image5":imread("4kImages/4k-3840-x-2160-wallpapers-themefoxx (5).jpg",cv2.IMREAD_GRAYSCALE),
    "Image6":imread("4kImages/4k-3840-x-2160-wallpapers-themefoxx (6).jpg",cv2.IMREAD_GRAYSCALE),
    "Image7":imread("4kImages/4k-3840-x-2160-wallpapers-themefoxx (7).jpg",cv2.IMREAD_GRAYSCALE),
    "Image8":imread("4kImages/4k-3840-x-2160-wallpapers-themefoxx (8).jpg",cv2.IMREAD_GRAYSCALE),
    "Image9":imread("4kImages/4k-3840-x-2160-wallpapers-themefoxx (9).jpg",cv2.IMREAD_GRAYSCALE),
    "Image10":imread("4kImages/4k-3840-x-2160-wallpapers-themefoxx (10).jpg",cv2.IMREAD_GRAYSCALE)
}

# Performance and Evaluation of SVD on 4k images

In [9]:
def compress_svd_images(Image,K):
    """
     compresses gray scale images and display the reconstructed image.
     Also displays a plot of singular values
    """
    image=data_images[Image]
    svd_img = compress_svd(image,K)
    
    cr_svd,original_size,compressed_size = compressionratio(image,svd_img)
    mse_svd = round(get_mse(image,svd_img),2)
    psnr_svd = round(get_psnr(image,svd_img),2)
    ssim_svd = round(get_ssim(image,svd_img),2)
    
    f, axarr = plt.subplots(1,2)
    axarr[0].imshow(image,cmap='gray')
    axarr[1].imshow(svd_img,cmap='gray')
    axarr[0].axis('off')
    axarr[1].axis('off')
    axarr[0].title.set_text(r"$\bf{Original\:Image}$"+"\nSize = {} KB\n\n\n\n".format(round(original_size/1024)))
    axarr[1].title.set_text(r"$\bf{SVD\:Image}$"+"\nSize = {} KB\nCompression Ratio = {}\nMean Squared Error = {}\nPeak Signal to Noise Ration = {}\nStructural Similarity Index Measure = {}".format(round(compressed_size/1024),cr_svd,mse_svd,psnr_svd,ssim_svd))

In [10]:
list_widget = widgets.Dropdown(options=list(data_images.keys()))
int_slider_widget = widgets.IntSlider(value=30,min=10,max=50)
interact(compress_svd_images,Image=list_widget,K=int_slider_widget);

interactive(children=(Dropdown(description='Image', options=('Image1', 'Image2', 'Image3', 'Image4', 'Image5',…

# Performance and Evaluation of DCT on 4k images

In [11]:
def compress_dct_images(Image,Threshold):
    image = data_images[Image]
    dct_img = compress_dct(image,Threshold/100)
    
    cr_dct,original_size,compressed_size = compressionratio(image,dct_img)
    mse_dct = round(get_mse(image,dct_img),2)
    psnr_dct = round(get_psnr(image,dct_img),2)
    ssim_dct = round(get_ssim(image,dct_img),2)
    
    f, axarr = plt.subplots(1,2)
    axarr[0].imshow(image,cmap='gray')
    axarr[1].imshow(dct_img,cmap='gray')
    axarr[0].axis('off')
    axarr[1].axis('off')
    axarr[0].title.set_text(r"$\bf{Original\:Image}$"+"\nSize = {} KB\n\n\n\n".format(round(original_size/1024)))
    axarr[1].title.set_text(r"$\bf{DCT\:Image}$"+"\nSize = {} KB\nCompression Ratio = {}\nMean Squared Error = {}\nPeak Signal to Noise Ration = {}\nStructural Similarity Index Measure = {}".format(round(compressed_size/1024),cr_dct,mse_dct,psnr_dct,ssim_dct))
    

In [12]:
list_widget = widgets.Dropdown(options=list(data_images.keys()))
int_slider_widget = widgets.FloatSlider(value=0.17,min=0.10,max=0.50)
interact(compress_dct_images,Image=list_widget,Threshold=int_slider_widget);

interactive(children=(Dropdown(description='Image', options=('Image1', 'Image2', 'Image3', 'Image4', 'Image5',…

In [13]:
def compare_dct_svd_images(Image):
    image = data_images[Image]
    svd_img = compress_svd(image,30)
    
    dct_img = compress_dct(image,0.00165)
    
    cr_svd,original_size,size_svd = compressionratio(image,svd_img)
    mse_svd = round(get_mse(image,svd_img),2)
    psnr_svd = round(get_psnr(image,svd_img),2)
    ssim_svd = round(get_ssim(image,svd_img),2)
    
    cr_dct,original_size,size_dct = compressionratio(image,dct_img)
    mse_dct = round(get_mse(image,dct_img),2)
    psnr_dct = round(get_psnr(image,dct_img),2)
    ssim_dct = round(get_ssim(image,dct_img),2)
    
    f, axarr = plt.subplots(1,3)
    axarr[0].imshow(image,cmap='gray')
    axarr[1].imshow(svd_img,cmap='gray')
    axarr[2].imshow(dct_img,cmap='gray')
    axarr[0].axis('off')
    axarr[1].axis('off')
    axarr[2].axis('off')
    axarr[0].title.set_text(r"$\bf{Original\:Image}$"+"\nSize = {} KB\n\n\n\n".format(round(original_size/1024)))
    axarr[1].title.set_text(r"$\bf{SVD\:Image}$"+"\nSize = {} KB\nCompression Ratio = {}\nMean Squared Error = {}\nPeak Signal to Noise Ration = {}\nStructural Similarity Index Measure = {}".format(round(size_svd/1024),cr_svd,mse_svd,psnr_svd,ssim_svd))
    axarr[2].title.set_text(r"$\bf{DCT\:Image}$"+"\nSize = {} KB\nCompression Ratio = {}\nMean Squared Error = {}\nPeak Signal to Noise Ration = {}\nStructural Similarity Index Measure = {}".format(round(size_dct/1024),cr_dct,mse_dct,psnr_dct,ssim_dct))
    plt.axis('off')

In [14]:
list_widget = widgets.Dropdown(options=list(data_images.keys()))
interact(compare_dct_svd_images,Image=list_widget);

interactive(children=(Dropdown(description='Image', options=('Image1', 'Image2', 'Image3', 'Image4', 'Image5',…